# Packaging and Deployment

In this exercise we will be covering the basic components of the flower federated learning package. We will build a basic client, that can interface with our live server. We will also package the client.

We will complete the following steps:  

    1.  Write a basic client
    2.  Dockerize the client
    3.  Deploy and run federated learning rounds with (hopefully) all workshop participants!

## Writing a basic client

The client is flower's abstract class that executes the code on the edge. It runs code to train the local model, updates the model parameters from parameters received from the server and runs evaluations on local data. Because the client is an abstract class, some methods do not have a default implementation. These methods are the methods that define how the local model is trained and how parameter updates get sent from the edge to the server.



**def get_parameters(self, config) -> List:** defines how parameters are extracted from the edge model and how they are sent back to the server. Our server already exists, and expects parameters in a two-dimensional list format, with the format being \[coefficients (NDArrays), intercept (NDArrays)\]. The config parameter is mandatory in this method, but we will not use it in our implementation. Configuration parameters requested by the server. This can be used to tell the client which parameters are needed along with some Scalar attributes.

**def fit(self, parameters, config): -> List, Integer, Dict** defines how the model will be refit, using the parameters passed by the server. The parameters argument is passed by the global (server) model. Configuration parameters allow the server to influence training on the client. It can be used to communicate arbitrary values from the server to the client, for example, to set the number of (local) training epochs. It returns the following three outputs: 

    parameters (NDArrays) – The locally updated model parameters.
    num_examples (int) – The number of examples used for training.
    metrics (Dict[str, Scalar]) – A dictionary mapping arbitrary string keys to values of type bool, bytes, float, int, or str. It can be used to communicate arbitrary values back to the server.

**def evaluate(self, parameters, config):** Evaluate the provided parameters using the locally held dataset. Returns these outputs:

    loss (float) – The evaluation loss of the model on the local dataset.
    num_examples (int) – The number of examples used for evaluation.
    metrics (Dict[str, Scalar]) – A dictionary mapping arbitrary string keys to values of type bool, bytes, float, int, or str. It can be used to communicate arbitrary values back to the server.

We have already written a client class below, with the skeleton methods. Implement these methods, and when you're down scroll down this notebook to find the second exercise.

In [2]:
from sklearn.linear_model import LinearRegression
import numpy as np
import flwr as fl
from sklearn.datasets import fetch_california_housing
from workshop.client.client import partition
from workshop.client.utils import set_initial_params

from typing import List


class CaliforniaHousingClient(fl.client.NumPyClient):
    def __init__(self, partition_id: int):
        self.data = None
        self.target = None
        self.model = LinearRegression()
        set_initial_params(self.model)

        X, y = fetch_california_housing(return_X_y=True)

        partition_id = np.random.choice(10)

        self.X_train, self.y_train = X[:15000], y[:15000]
        self.X_test, self.y_test = X[15000:], y[15000:]

        self.X_train, self.y_train = partition(
            self.X_train, self.y_train, 10)[partition_id]
        
    def get_parameters(self, config) -> List:
        """Returns the paramters of a sklearn LinearRegression model."""

    def fit(self, parameters, config):
        """Refit the model locally with the central parameters and return them"""
        updated_parameters = parameters
        num_examples = 0
        metrics = {} # Won't be used in this example, we can return it empty

        return updated_parameters, num_examples, metrics


ModuleNotFoundError: No module named 'workshop'